In [1]:
import sys
sys.path.append('D:\PyCharmProjects\VFPUMC02')
sys.path.append(r'C:\Users\Administrator\PycharmProjects\VFPUMC02')
sys.path.append(r'/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

In [27]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import split_labeled_unlabeled
from utils.DataProcessUtils import find_rounds_math
from utils.FateUtils import evaluate_model

In [3]:
bankDataset = BankDataset()

2025-01-21 12:33:07,376 - INFO - 正在加载数据集 bank.csv
2025-01-21 12:33:07,602 - INFO - 数据集 bank.csv 加载完成，数据集形状为 (41188, 64)


In [4]:
X, y = bankDataset.get_data()

In [5]:
X_L, y_L, X_U, y_U = split_labeled_unlabeled(X, y, k=0.2)

原始数据 X 形状: (41188, 63)
原始标签 y 形状: (41188,)
选择 8237 个样本作为有标签数据，剩余样本作为无标签数据
有标签数据 X_L 形状: (8237, 63)
有标签数据 y_L 形状: (8237,)
无标签数据 X_U 形状: (32951, 63)
无标签数据 y_U 形状: (32951,)


In [6]:
config_clf = {
    'loss': 'log_loss',
    'learning_rate': 0.1,
    'n_estimators': 100
}

config_reg = {
    'loss': 'squared_error',
    'learning_rate': 0.1,
    'n_estimators': 100
}

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

In [8]:
clf = GradientBoostingClassifier(**config_clf)

In [9]:
reg = GradientBoostingRegressor(**config_reg)

In [10]:
from classfiers.TwoStep import TwoStep

In [17]:
max_iter = find_rounds_math(len(X_U), 0.1, 0)

In [19]:
two_step = TwoStep(base_classifier=clf, base_regressor=reg,max_iter=100,k=0.1)

2025-01-21 12:36:41,053 - INFO - TwoStep 类已初始化: max_iter=100, k=0.10, base_classifier=GradientBoostingClassifier, base_regressor=GradientBoostingRegressor


In [20]:
two_step.fit(X_L, y_L, X_U)

2025-01-21 12:37:26,191 - INFO - 有标签数据 (L): 8237 样本
2025-01-21 12:37:26,192 - INFO - 无标签数据 (U): 32951 样本
2025-01-21 12:37:26,192 - INFO - 任务类型判定结果: classification
2025-01-21 12:37:26,193 - INFO - 这是一个分类任务。
2025-01-21 12:37:26,194 - INFO - 分类任务中的类别: [0 1]
2025-01-21 12:37:26,195 - INFO - 初始化预测结果数组，长度=32951
2025-01-21 12:37:26,195 - INFO - 开始自训练迭代，共计划进行 100 次迭代
2025-01-21 12:37:26,196 - INFO - ===== 第 0 次迭代开始 =====
2025-01-21 12:37:26,196 - INFO - 当前 Labeled 数据量: 8237；Unlabeled 数据量: 32951
2025-01-21 12:37:26,196 - INFO - 开始训练分类器 (基于 8237 个样本)...
2025-01-21 12:37:27,721 - INFO - 分类器训练完成，耗时 1.52 秒。
2025-01-21 12:37:27,722 - INFO - 分类器识别的类别: [0 1]
2025-01-21 12:37:27,722 - INFO - 开始对 Unlabeled 数据进行置信度打分...
2025-01-21 12:37:27,778 - INFO - 完成置信度打分，耗时 0.06 秒。
2025-01-21 12:37:27,779 - INFO - 选取前 10.00% 的高置信度样本。
2025-01-21 12:37:27,779 - INFO - 本轮选出高置信度样本数量: 3295，占比约为 10.00%
2025-01-21 12:37:27,780 - INFO - 对选中的高置信度样本进行预测标签。
2025-01-21 12:37:27,829 - INFO - 将高置信度样本合并到 Labeled 集。
2025-01-21 12:

In [22]:
y_pred = two_step.get_unlabled_predict()

In [26]:
y_pred = y_pred.astype('int64')

In [28]:
evaluate_model(y_true=y_U,y_pred=y_pred)


========== Classification Metrics ==========
Accuracy         : 0.9124
Precision (macro): 0.7853
Recall (macro)   : 0.7634
F1 (macro)       : 0.7737

Confusion Matrix:
[[27933  1280]
 [ 1605  2133]]

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     29213
           1       0.62      0.57      0.60      3738

    accuracy                           0.91     32951
   macro avg       0.79      0.76      0.77     32951
weighted avg       0.91      0.91      0.91     32951



----

回归测试

In [30]:
X, y  = X[:,:-1],X[:,-1]

In [33]:
X_L, y_L, X_U, y_U = split_labeled_unlabeled(X, y, k=0.2)

原始数据 X 形状: (41188, 62)
原始标签 y 形状: (41188,)
选择 8237 个样本作为有标签数据，剩余样本作为无标签数据
有标签数据 X_L 形状: (8237, 62)
有标签数据 y_L 形状: (8237,)
无标签数据 X_U 形状: (32951, 62)
无标签数据 y_U 形状: (32951,)


In [34]:
two_step.fit(X_L, y_L, X_U)

2025-01-21 14:03:34,540 - INFO - 有标签数据 (L): 8237 样本
2025-01-21 14:03:34,541 - INFO - 无标签数据 (U): 32951 样本
2025-01-21 14:03:34,541 - INFO - 任务类型判定结果: regression
2025-01-21 14:03:34,541 - INFO - 这是一个回归任务。
2025-01-21 14:03:34,542 - INFO - 回归任务标签统计: 最小值=-2.8157, 最大值=0.8452, 平均值=-0.0026, 标准差=0.9926
2025-01-21 14:03:34,543 - INFO - 初始化预测结果数组，长度=32951
2025-01-21 14:03:34,543 - INFO - 开始自训练迭代，共计划进行 100 次迭代
2025-01-21 14:03:34,544 - INFO - ===== 第 0 次迭代开始 =====
2025-01-21 14:03:34,544 - INFO - 当前 Labeled 数据量: 8237；Unlabeled 数据量: 32951
2025-01-21 14:03:34,545 - INFO - 开始训练回归器 (基于 8237 个样本)...
2025-01-21 14:03:35,873 - INFO - 回归器训练完成，耗时 1.33 秒。
2025-01-21 14:03:35,874 - INFO - 开始对 Unlabeled 数据进行置信度打分...
2025-01-21 14:03:35,938 - INFO - 完成置信度打分，耗时 0.06 秒。
2025-01-21 14:03:35,939 - INFO - 选取前 10.00% 的高置信度样本。
2025-01-21 14:03:35,940 - INFO - 本轮选出高置信度样本数量: 3295，占比约为 10.00%
2025-01-21 14:03:35,940 - INFO - 对选中的高置信度样本进行预测标签。
2025-01-21 14:03:35,941 - INFO - 将高置信度样本合并到 Labeled 集。
2025-01-21 14:03:35,943 

In [35]:
y_pred = two_step.get_unlabled_predict()

In [38]:
evaluate_model(y_true=y_U,y_pred=y_pred,task='regression')


========== Regression Metrics ==========
MSE               : 0.0000
RMSE              : 0.0058
MAE               : 0.0012
R^2 (Coefficient) : 1.0000
Explained Variance: 1.0000


In [44]:
import numpy as np
t = np.full(1000, 1.0)

In [46]:
t.dtype

dtype('float64')

In [48]:
t.astype('int').dtype

dtype('int64')